# Running HR Model Single Inference on Edge TPU

The following set of codes is for running a single inference of the model on Edge TPU. <br> 
One image is selected as input for the model and then the result is displayed back after execution. <br> <br>
Note on Usage:
This file is to be executed on an Edge TPU, typically the Coral Dev Board. A jupyter notebook has been<br> 
installed on the Coral Board under user 'Mendel'. Type 'jupyter notebook' or 'jupyter lab' in the linux<br>
command prompt to open the code files.

### Importing necessary dependencies, including Pycoral API

For this the pycoral API has to be installed. <link>https://coral.ai/docs/reference/py/</link> <br> 
Pycoral API and the necessary dependencies are already installed on the Coral Board ("tuned shrimp") <br> 
under the user 'Mendel'.

In [ ]:
import os
import cv2 as cv
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from pycoral.utils import edgetpu
from pycoral.utils import dataset
from pycoral.adapters import common
from pycoral.adapters import classify

### Specifying the model, labels and images

Here the model specified has to be compatible with Edge TPU. Ensure the model is converted, quantized <br>
and compiled with the Edgetpu compiler. The model file name should have 'edgetpu.tflite' in the end.

In [ ]:
model_file = 'hr_model_quant_edgetpu.tflite' # model edgetpu-tflite file
label_file = 'digits_labels.txt' # file with labels for classes
image_file = '2_img.GIF' # chose your image file here for inference

### Building the interpreter

In [ ]:
interpreter = edgetpu.make_interpreter(model_file)
interpreter.allocate_tensors()

In [ ]:
# Checks if the model is integer quantized or not
if common.input_details(interpreter, 'dtype') != np.uint8:
    raise ValueError('Only support uint8 input type.')
else:
    print('model is quantized, ready to use')

### Processing the input image

In [ ]:
# finding the image size required from the interpreter
size = common.input_size(interpreter)
#image = Image.open(image_file).convert('L').resize(size, Image.ANTIALIAS)

# resizing the image with greyscale, inverting and converting it into numpy array
proc_img = ImageOps.invert(ImageOps.grayscale(Image.open(image_file).convert('RGB').resize(size, Image.ANTIALIAS)))
img_array = np.array(proc_img)
print(img_array.shape) # get the shape and verify if it fits with the model input size

In [ ]:
# plotting the images for visualization
fig = plt.figure(figsize=(7, 3))

fig.add_subplot(1, 2, 1)
plt.imshow(Image.open(image_file))
plt.axis('off')
plt.title("Raw Image")

fig.add_subplot(1, 2, 2)
plt.axis('off')
plt.title("Processed Image")
plt.imshow(img_array, cmap='gray')

In [ ]:
# first broadcasting the img_array to fit the dimentions (28, 28, 1) in this case
# it depends on your model input required dimension whether you have to broadcast 
# the array or not
# this is the recommended way, the alternative manual method is given below
img_array = img_array.reshape((28,28,1))
print(img_array.shape)

In [ ]:
# Manual image preprocessing - 2 transformations: normalization and quantization
# combined in 1 line: q = (input - mean) / (std * scale) + zero_point
# 1e-5 used instead of 0 to prevent division by 0 error

# defining the parameters
top_k = 1
threshold = 0.0
count = 1
input_mean = 128.0
input_std = 128.0

params = common.input_details(interpreter, 'quantization_parameters')
print(params)
scale = params['scales']
zero_point = params['zero_points']

if abs(scale * std - 1) < 1e-5 and abs(mean - zero_point) < 1e-5:
    # Input data does not require preprocessing.
    print('no preprocessing required')
    common.set_input(interpreter, image)
else:
    # Input data requires preprocessing
    print('image preprocessed')
    img_array = img_array.reshape((28,28,1))
    normalized_input = (img_array - mean) / (std * scale) + zero_point
    np.clip(normalized_input, 0, 255, out=normalized_input)
    common.set_input(interpreter, normalized_input.astype(np.uint8))


### Running the Inference and Printing the Result

In [ ]:
# Running the inference
common.set_input(interpreter, img_array)
interpreter.invoke()
classes = classify.get_classes(interpreter, top_k=1)
print(classes) # this prints the resultant class id chosen by the model

In [ ]:
# Printing the result
# use this if you have a labels file
labels = dataset.read_label_file(label_file)
print("class | score")
for c in classes:
    print('%s     | %.5f' % (labels.get(c.id, c.id), c.score))
    #print(c.score)